In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


### MNIST con K-vecinos más vercanos
En esta segunda sesión práctica vamos a entrenar clasificadores basados en distancias.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)



(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Solución

x_train, x_test, y_train, y_test  = train_test_split(data, targets, test_size=0.1, random_state=1)

part=(7000.0)/(70000.0-7000)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=part, random_state=1) 

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(56000, 784)
(7000, 784)
(7000, 784)


### **Ejercicio 2**   

Prueba un clasificador KNeighborsClassifier

In [4]:
## Solución
kv = KNeighborsClassifier()
kv.fit(x_train,y_train)
    
ypred = kv.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(acc)

0.9732857142857143


### **Ejercicio 3**   

Encuentra el mejor valor de "K" con el conjunto de validación y obtén el resultados para test.

In [5]:
## Solución

max=-1
for k in [1,3,4,5,6,7,8,9,10,15,20,25,30]:
    kv = KNeighborsClassifier(n_neighbors=k)
    kv.fit(x_train,y_train)
    
    ypred = kv.predict(x_val)
    acc = accuracy_score(y_val, ypred)
    print(k,acc)
    if acc>max: 
        max=acc
        best=k

print("=================")
print("BEST",best,max)
    


1 0.9738571428571429
3 0.976
4 0.9744285714285714
5 0.9732857142857143
6 0.973
7 0.9725714285714285
8 0.9718571428571429
9 0.9717142857142858
10 0.9712857142857143
15 0.9677142857142857
20 0.9674285714285714
25 0.9654285714285714
30 0.9632857142857143
BEST 3 0.976


In [6]:
## Solución

data_tr=np.concatenate((x_train,x_val),axis=0)
targets_tr=np.concatenate((y_train,y_val),axis=0)

kv = KNeighborsClassifier(n_neighbors=best)
kv.fit(data_tr,targets_tr)

ypred = kv.predict(x_test)
acc = accuracy_score(y_test, ypred)
print(acc)



0.9718571428571429


## **Ejercicio 4**

Cambia la distancia que se emplea (por defecto la L2) por la L1. Ver la documentación para ver cómo poder definir funciones de distancia y pasarlas como parámetro. Esto ralentiza bastante la clasificación pero se puede emplear el parámetro "n_jobs" para paralelizar el proceso. Aún así el resultado es muy lento. Podríamos concluir que sk-learn no es una librería adecuada para clasificar con KNN con métricas definidas por el usuario. Por ejemplo no permite definir un algoritmo rápido de tipo kd_tree.

In [7]:
## Solución

def distancia(x,y):
    return np.sum(np.abs(x-y))

kv = KNeighborsClassifier(n_neighbors=3,metric=distancia,n_jobs=4)
kv.fit(data_tr,targets_tr)

print("predict")
ypred = kv.predict(x_test[0:100,:])
acc = accuracy_score(y_test[0:100], ypred[0:100])
print(acc)

predict
0.94
